<h1>Powering Energy Storage Beyond Excel</h1>
<h2>This is a sample document to show the power of python over Excel files</h2>

In [73]:
# Required Dependencies
import tabulate
import pandas as pd
from decimal import Decimal as D

In [74]:
def convert_data_to_df(file, columns, skiprows):
    """
    Read and excel file with specific columns and skip any non required 
    rows you encounter during reading rows.
    """
    specifications = pd.read_excel(file, skiprows=skiprows, usecols=columns)
    return specifications

<h3>Reading Data from excel to Store in Database for further use in other components</h3>

<h3>Storing Constant Data for Project</h3>

In [75]:
def convert_data_type(data_property):
    """
    Convert list of data into a dictionary, where keys
    represent the years and values represent the values
    against that year. 
    """
    output = {
        index: D(value)
            for index, value in enumerate(data_property, start=1)
    }
    return output

In [76]:
def get_constant_data_for_project(sheet_name, columns, skiprows):
    """
    Read only specific columns from the excel file.
    """
    data = convert_data_to_df(sheet_name, columns=columns, skiprows=skiprows).iloc[0:40]
    return data


In [77]:
sheet_data = get_constant_data_for_project("data/sample.xlsx", columns="T:X", skiprows=54)

"""
Get required columns from the data frame and convert them into 
desired behavior using convert_data_type() function
"""

PERFORMANCE_DETERIORATION_PROFILE =  convert_data_type(sheet_data["Performance Deterioration Profile"])
STEPWISE_DIRECT_CURRENT_GENERATION_INCREASE = convert_data_type(sheet_data["Stepwise Direct Current Generation Increase"])
DETERIORATION_CURVE_FOR_PHASE_GREATER_THAN1 = convert_data_type(sheet_data["Deterioration Curve for Phase Greater Than 1"])
PROGRESSIVE_PROJECT_DIRECT_CURRENT_GENERATION = convert_data_type(sheet_data["Progressive Project Direct Current Generation"])
PERFORMANCE_DECAY_WITH_PERFORMANCE_GRID_MATRIX = convert_data_type(sheet_data["Performance Decay with Performance Grid Matrix"])

<h3>Implementing calculations and formulas from an excel file</h3>

In [78]:
class Phase:
    """Calculate PG Table Phase formulas for the project."""

    def __init__(self, phase_info):
        """Constructor"""
        self.phase_info = phase_info
        self.years = [year for year in range(1, 41)]
        self.phase_degradation_with_deterioration_profile = (
            PERFORMANCE_DETERIORATION_PROFILE
            if self.phase_info == 1
            else DETERIORATION_CURVE_FOR_PHASE_GREATER_THAN1
        )

    @property
    def incremental_phase_dc_generation(self):
        """
        Incremental Phase DC Generation
        """
        return STEPWISE_DIRECT_CURRENT_GENERATION_INCREASE

    @property
    def total_phase_dc_generation(self):
        """
        Total Phase DC Generation
        =SUM(Hn:H$n)
        """
        # n is a natural number
        incremental_phase_dc_generation = self.incremental_phase_dc_generation
        result = {1: 0, 2: incremental_phase_dc_generation[2]}
        for year in self.years[2:]:
            result[year] = sum(
                [incremental_phase_dc_generation[prev_year] for prev_year in result]
                + [incremental_phase_dc_generation[year]]
            )

        return result

    @property
    def phase_years(self):
        """
        Total Years for a project
        """
        return self.years

    @property
    def performance_decay_with_performance_grid_matrix(self):
        """
        A constant value throughout the project
        """
        return PERFORMANCE_DECAY_WITH_PERFORMANCE_GRID_MATRIX

    @property
    def performance_decay_with_performance_grid_matrix_delta(self):
        """
        Deg w/ PG Matrix vs HARD INPUT SOH Delta
        """
        performance_decay_with_performance_grid_matrix = self.performance_decay_with_performance_grid_matrix
        result = {}
        for year in self.years:
            result[year] = abs(
                round(performance_decay_with_performance_grid_matrix[year], 6)
                - round(D(self.phase_degradation_with_deterioration_profile[year]), 6)
            )
        return result


In [79]:
class PerformanceTableProject:
    """Performance Table formulas"""

    phase_1 = None
    phase_2 = None

    def __init__(self):
        self.years = [year for year in range(1, 41)]
        self.initialize_phases()

    def initialize_phases(self):
        for phase in [phase for phase in range(1, 3)]:
            setattr(self, f"phase_{phase}", Phase(phase_info=phase),)

    @property
    def project_performance_decay_with_performance_grid_matrix_delta(self):
        """Calculate Deg w/ PG Matrix vs HARD INPUT SOH Delta"""
        phase_deg_pg_matrix = {}
        for phase in [phase for phase in range(1, 3)]:
            phase_obj = getattr(self, f"phase_{phase}")
            phase_deg_pg_matrix[phase] = phase_obj.performance_decay_with_performance_grid_matrix_delta

        result = {}
        for year in self.years:
            result[year] = max(
                [phase_deg_pg_matrix[phase][year] for phase in [phase for phase in range(1, 3)]]
            )

        return result

    @property
    def project_cycles(self):
        result = {1: 0}
        for year in self.years[1:]:
            result[year] = result[year - 1] + self.cycles_per_year
        return result


In [80]:
performance_table = PerformanceTableProject()
table_output = performance_table.project_performance_decay_with_performance_grid_matrix_delta

<h3>Compare results from Excel sheet and Code</h3>

In [81]:
output_data = convert_data_to_df("data/sample.xlsx", columns="M", skiprows=50)

output_soh_delta = output_data["Performance Decay with Performance Grid Matrix vs. Initial State of Health (ISOH) Delta"]

data = [
    {"Excel": output_soh_delta[year-1], "Python": value} for year, value in table_output.items()
]
table = tabulate.tabulate(data, tablefmt="html", headers="keys")
table

Excel,Python
0,0
0.0703328,0.070333
0.102085,0.102085
0.115988,0.115988
0.127193,0.127193
0.140799,0.140799
0.15302,0.15302
0.164308,0.164308
0.177549,0.177549
0.193524,0.193524
